# Voice Activity

In [1]:
import numpy as np
import librosa
import librosa.display
import os
import scipy.io.wavfile
%matplotlib inline

### Relative Path

In [11]:
cur_path = os.getcwd()
relative_path = os.path.relpath('..\\data\\pepper robot sesleri\\raw\\Copy of Aferin.wav', cur_path)
recordNames = os.listdir("./ses/")
recordNames

['ENG',
 '4CH072I',
 '4CH071I',
 '.DS_Store',
 '4CH069I',
 '4CH067I',
 'MAAP3',
 'MAAP5',
 'MAAP2',
 '4CH065I',
 '4CH066I',
 '4CH068I',
 'MAAP7',
 'MAAP6']

In [3]:
def split_into_intervals(data, k = 1600):
    intervals = [data[n*k:(n+1)*k] for n in range(int(len(data)/k))]
    return intervals

## 1. Remove silence.

In [4]:
def split_and_export(path_in, path_out, k, title, remove_silence = 1):
    if not os.path.isdir(path_out):
        os.mkdir(path_out)
    audioFiles = [f for f in os.listdir(path_in) if os.path.isfile(os.path.join(path_in, f))]
    mergedAudioData = []
    for audio in audioFiles:
        if ".wav" not in audio:
            continue
        data, sr = librosa.load(path_in + "/" + audio, sr=8000)
        mergedAudioData.extend(data)
    if remove_silence:
        silenceRemovedData, index = librosa.effects.trim(np.array(mergedAudioData), top_db=50)
        robot_intervals = split_into_intervals(np.array(silenceRemovedData), k)
    else:
        robot_intervals = split_into_intervals(np.array(mergedAudioData), k)
    counter = 0
    for i in range(0, len(robot_intervals)):
        scipy.io.wavfile.write(os.path.abspath(path_out) + "/" +"Sample{}{}_K{}.wav".format(title, i, k), sr, robot_intervals[i])

In [7]:
notSampledPath = "./ses/{}/{}"
sampledPath = "./ses/{}/samples_{}"

In [12]:
for recordName in recordNames:
    if "DS" not in recordName:
        for label in [0,1]:
            relative_path_in = os.path.relpath(notSampledPath.format(recordName, label), cur_path)
            relative_path_out = os.path.relpath(sampledPath.format(recordName, label), cur_path)
            split_and_export(relative_path_in, relative_path_out, 1600, f"enes_{label}label_", remove_silence = label)